# Example usage

To use `labtools` in a project:

## Imports

In [1]:
from labtools import shuffle

## Shuffle a sequence

Create shuffles of the alphabet:

In [2]:
shuffles_list, names_list = shuffle.windowed_shuffle("ABCDEFGHIJKLMNOPQRSTUVWYXZ")
shuffles_list

['bedacFGHIJKLMNOPQRSTUVWYXZ',
 'AcdfebGHIJKLMNOPQRSTUVWYXZ',
 'ABcegfdHIJKLMNOPQRSTUVWYXZ',
 'ABChfgedIJKLMNOPQRSTUVWYXZ',
 'ABCDfgiheJKLMNOPQRSTUVWYXZ',
 'ABCDEifghjKLMNOPQRSTUVWYXZ',
 'ABCDEFhgikjLMNOPQRSTUVWYXZ',
 'ABCDEFGkijhlMNOPQRSTUVWYXZ',
 'ABCDEFGHjkilmNOPQRSTUVWYXZ',
 'ABCDEFGHIlnjmkOPQRSTUVWYXZ',
 'ABCDEFGHIJonmlkPQRSTUVWYXZ',
 'ABCDEFGHIJKnlmpoQRSTUVWYXZ',
 'ABCDEFGHIJKLpmnqoRSTUVWYXZ',
 'ABCDEFGHIJKLMpnorqSTUVWYXZ',
 'ABCDEFGHIJKLMNrospqTUVWYXZ',
 'ABCDEFGHIJKLMNOqprstUVWYXZ',
 'ABCDEFGHIJKLMNOPrutsqVWYXZ',
 'ABCDEFGHIJKLMNOPQstuvrWYXZ',
 'ABCDEFGHIJKLMNOPQRwusvtYXZ',
 'ABCDEFGHIJKLMNOPQRSuwtvyXZ',
 'ABCDEFGHIJKLMNOPQRSTxvyuwZ',
 'ABCDEFGHIJKLMNOPQRSTUxvzyw']

In [3]:
from labtools.adtools import seqlib

In [4]:
from labtools.adtools.finder import pull_AD

In [5]:
pull_AD("ACTTTTATVGCTAGCATGGCTGGTAGATCTTGGTTGATTGATTCTAATAGAATTGCTACTAAGATTATGTCTGCTTCTGCTTCTTCTGATCCAAGACAAGTTGTTTGGAAATCTAATCCATCTAGACATTGTCCAGCTGATCGATGCTAGTAGAGAGAGA")

('ATGGCTGGTAGATCTTGGTTGATTGATTCTAATAGAATTGCTACTAAGATTATGTCTGCTTCTGCTTCTTCTGATCCAAGACAAGTTGTTTGGAAATCTAATCCATCTAGACATTGTCCA',
 None)

In [6]:
from labtools.adtools.pair_counter import seq_counter
seq_counter("../exampledata/mini.fastq")

GGTTCTTCTAAATTGAGATGTGATAATAATGCTGCTGCTCATGTTAAATTGGATTCATTTCCAGCTGGTGTTAGATTTGATACATCTGATGAAGAATTGTTGGAACATTTGGCTGCTAAA    1
GAAGAATTGTTTTTACATTTGTCTGCTAAGATTGGTAGATCTTCTAGGAAACCACATCCATTCTTGGATGAATTTATTCATACTTTGGTTGAAGAAGATGGTATTTGTAGAACTCATCCA    3
dtype: int64